# Homework 1

## Q1: Which is the build_hash value?

In [22]:
!curl localhost:9200 #The version build_hash is 42f05b9372a9a4a470db3b52817899b99a76ee73 

{
  "name" : "def45470fc11",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "nFldxZjaSKKHkt4xxIKLlQ",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


## Q2: Index the data in the ame way was done in the videos. Which is the function to add data to  elastic?
The function is index

In [16]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [20]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200')

In [24]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}
index_name = 'course-questions'

es_client.indices.create(index=index_name,body=index_settings)

from tqdm.auto import tqdm
for doc in tqdm(documents):
    es_client.index(index = index_name,document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

## Q3 now lets exectue the query 'How do I execute a command in a running docker container'? Use only question and text fields and give question a boost of 4 and use type : best fields


In [31]:
query = 'How do I execute a command in a running docker container'
search_query = {
    "size": 1,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"], 
                    "type": "best_fields"
                }
            }
        }
    }
}

response = es_client.search(index=index_name,body=search_query)

In [32]:
response['hits']

{'total': {'value': 865, 'relation': 'eq'},
 'max_score': 84.050095,
 'hits': [{'_index': 'course-questions',
   '_id': 'WsGnU5UB9SkX3tb9zGa-',
   '_score': 84.050095,
   '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
    'section': '5. Deploying Machine Learning Models',
    'question': 'How do I debug a docker container?',
    'course': 'machine-learning-zoomcamp'}}]}

## Q4: Now lets only limit the questions to machine-learning-zoomcamp
### Return 3 results 

In [49]:
search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"], 
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

query = 'How do I execute a command in a running docker container'
response = es_client.search(index=index_name,body = search_query)
response['hits']

{'total': {'value': 300, 'relation': 'eq'},
 'max_score': 103.06507,
 'hits': [{'_index': 'course-questions',
   '_id': 'WsGnU5UB9SkX3tb9zGa-',
   '_score': 103.06507,
   '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
    'section': '5. Deploying Machine Learning Models',
    'question': 'How do I debug a docker container?',
    'course': 'machine-learning-zoomcamp'}},
  {'_index': 'course-questions',
   '_id': 'ecGnU5UB9SkX3tb9zWZE',
   '_score': 51.04628,
   '_source': {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use 

## Q5: Build a prompt now

In [50]:
context_template = """
Q: {question}
A: {text}
""".strip()

prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION:{question}

CONTEXT:
{context}
""".strip()

In [51]:
context = ""

for hit in response['hits']['hits']:
    context += context_template.format(text = hit['_source']['text'], question = hit['_source']['question'])
    context += '\n\n'

prompt = prompt_template.format(question = query, context = context)

In [52]:
print(len(prompt))

1396


## Q6: Tokens

In [53]:
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 56.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 796.9/796.9 kB 48.5 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [55]:
import tiktoken
encoding = tiktoken.encoding_for_model('gpt-4o')

In [57]:
len(encoding.encode(prompt))

306

In [58]:
encoding.decode_single_token_bytes(63842)

b"You're"